In [1]:
import uproot
import matplotlib
import matplotlib.pyplot as plt
import awkward as ak
import hist
import numpy as np
from event_display import gdml_read_ecal_info
import os
import pandas as pd

%matplotlib inline
matplotlib.rc("figure", dpi=200)

dir_path = os.path.dirname(os.path.realpath("__file__"))
print(f"Believed path is: {dir_path}")

Believed path is: /home/romanov/eic/kak-bog-cherepahu/simulation


In [2]:
# Data file name
#data_file_name = f"{dir_path}/data/test_gun.edm4hep.root"
file_event_num = 10000

# Number of events to process:
entry_start = 0                     # (!) Try changing those
entry_stop = file_event_num         # <==
events_to_read = entry_stop-entry_start

# base_name = 'disk_gun_piminus_0-15GeV'
base_name = '2022-11-22_pgun_pi-_wall_only_e0.01-10GeV_center_1prt_10000evt'
root_file_name = f"{dir_path}/{base_name}.edm4hep.root"
output_file = f'{dir_path}/{base_name}.npz'

In [3]:
# Open root file and get "events" tree from it
tree = uproot.open(root_file_name)["events"]

# Read fields from a file
gen_status = tree['MCParticles/MCParticles.generatorStatus'].array(entry_start=entry_start, entry_stop=entry_stop)
masses = tree['MCParticles/MCParticles.mass'].array(entry_start=entry_start, entry_stop=entry_stop)
px = tree['MCParticles/MCParticles.momentum.x'].array(entry_start=entry_start, entry_stop=entry_stop)
py = tree['MCParticles/MCParticles.momentum.y'].array(entry_start=entry_start, entry_stop=entry_stop)
pz = tree['MCParticles/MCParticles.momentum.z'].array(entry_start=entry_start, entry_stop=entry_stop)
pdg = tree['MCParticles/MCParticles.PDG'].array(entry_start=entry_start, entry_stop=entry_stop)
pos_x = tree['MCParticles/MCParticles.vertex.x'].array(entry_start=entry_start, entry_stop=entry_stop)
pos_y = tree['MCParticles/MCParticles.vertex.y'].array(entry_start=entry_start, entry_stop=entry_stop)
pos_z = tree['MCParticles/MCParticles.vertex.z'].array(entry_start=entry_start, entry_stop=entry_stop)

# 'stable' are particles from particle gun
stable_only = gen_status > 0

# filter other particles
masses = masses[stable_only]
px = px[stable_only]
py = py[stable_only]
pz = pz[stable_only]
pdg = ak.flatten(pdg[stable_only]).to_numpy()
pos_x = ak.flatten(pos_x[stable_only]).to_numpy()
pos_y = ak.flatten(pos_y[stable_only]).to_numpy()
pos_z = ak.flatten(pos_z[stable_only]).to_numpy()

# calculate energy
energies = np.sqrt(masses * masses + px * px + py * py + pz * pz)
energies = ak.flatten(energies).to_numpy()


In [4]:
assert len(pos_x)==events_to_read
assert len(pos_y)==events_to_read
assert len(energies)==events_to_read


In [5]:
# Load geometry file
ecal_info = gdml_read_ecal_info(f"{dir_path}/wall_only.gdml")

# ecal_info is of EcalGeoInfo class, which is a helper holding information
# about all needed ecal geometries.
# Print what information it holds:
ecal_info.print()

module_size_x    : 2.0
module_size_y    : 2.0
module_size_z    : 20.0
total_modules    : 169
num_modules_x    : 13
num_modules_y    : 13
min_x            : -12.299999999999999
max_x            : 12.299999999999999
min_y            : -12.299999999999999
max_y            : 12.299999999999999
min_z            : 0.0
max_z            : 0.0
border_left      : -13.299999999999999
border_right     : 13.299999999999999
border_top       : 13.299999999999999
border_bottom    : -13.299999999999999
unit             : cm


In [6]:
events = ecal_info.read_events_from_file(root_file_name, 0, file_event_num)

assert len(events) == len(energies)

In [7]:
original_shape = np.shape(events)
print(f"Original shape {original_shape}")

flatten_shape = (original_shape[0], original_shape[1] * original_shape[2])
print(f"Flatten shape {flatten_shape}")
reshaped_events = np.reshape(events, flatten_shape)

Original shape (10000, 13, 13)
Flatten shape (10000, 169)


In [8]:
masses = masses[stable_only]
px = px[stable_only]
py = py[stable_only]
pz = pz[stable_only]

In [9]:
np.savez_compressed(output_file, modules=reshaped_events, true_e=energies, true_x=pos_x, true_y=pos_y, true_pdg=pdg)